# Session 8 - Language modelling with RNNs (Text Generation)

In [1]:
# data processing tools
import string, os 
import pandas as pd
import numpy as np
np.random.seed(42)

# keras module for building LSTM 
import tensorflow as tf
tf.random.set_seed(42)
import tensorflow.keras.utils as ku 
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# surpress warnings
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

2023-03-22 15:20:00.424613: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Some helper functions

In [2]:
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower() # make all text lowercase, keep strings that arent punctuation
    txt = txt.encode("utf8").decode("ascii",'ignore') # utf8 encoding makes up for accents
    return txt 

def get_sequence_of_tokens(tokenizer, corpus):
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences

def generate_padded_sequences(input_sequences):
    # get the length of the longest sequence
    max_sequence_len = max([len(x) for x in input_sequences])
    # make every sequence the length of the longest on
    input_sequences = np.array(pad_sequences(input_sequences, 
                                            maxlen=max_sequence_len, 
                                            padding='pre'))

    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, 
                            num_classes=total_words)
    return predictors, label, max_sequence_len

def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 
                        10, 
                        input_length=input_len))
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1)) # remove 10% of the weight when the model is training
    # Add Output Layer
    model.add(Dense(total_words, 
                    activation='softmax'))
    model.compile(loss='categorical_crossentropy', 
                    optimizer='adam')
    return model

def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], 
                                    maxlen=max_sequence_len-1, 
                                    padding='pre')
        predicted = np.argmax(model.predict(token_list),
                                            axis=1)   
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

## Load the data

In [12]:
data_dir = os.path.join("../in/news_data")

We're then going to load the data one at a time and append *only* the headlines to our list of data.

In [13]:
all_comments = []
for filename in os.listdir(data_dir):
    if 'Comments' in filename:
        comment_df = pd.read_csv(data_dir + "/" + filename)
        all_comments.extend(list(comment_df["commentBody"].values))

We then clean up a little bit and see how many data points we have.

In [14]:
all_comments = [c for c in all_comments if c != "Unknown"] # remove all unknown headlines
len(all_comments)

203199

We call out ```clean_text()``` function and then inspect the first 10 texts.

In [15]:
corpus = [clean_text(x) for x in all_comments]
corpus[:10]

['i typically strongly dislike articles which bring up such tiny disadvantages to women but this article i liked it mentioned with many female politicians being replaced by male lawmakers who have pushed for legislation to limit womens access to abortion which would normally tick my nonfeminist self off in a regular old article but i feel as though the beating of their drums are beating a new perspective in my heart the bahia group is going against cultural norms by becoming the first allfemale blocoafro in brazil and that problem was certainly not tiny they are drumming for their rights as women they are drumming for their rights of their race this article has influenced me to stand for any matter i want to defend with determination just as these women did in banda dida',
 'i went to cuba twice in 19989 to study afrocuban drumming  i am a woman  i didnt experience any prejudicei suppose money silences thatbut we also visited matanzas where we listened to stellar drummers in the home o

## Tokenize

We're then going to tokenize our data, using the ```Tokenizer()``` class from ```TensorFlow```, about which you can read more [here](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer).

We then use the ```get_sequence_of_tokens()``` function we defined above, which turns every text into a sequence of tokens based on the vocabulary from the tokenizer.

In [16]:
# this code creates an index of the words in the corpus
tokenizer = Tokenizer()
## tokenization
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1 # +1 accounts for words that arent in the vocabulary. 

In [17]:
# take input and turn it into numerical output
inp_sequences = get_sequence_of_tokens(tokenizer, corpus)
inp_sequences[:10]
# the length of the input sequences are defined by how the words relate to each other

[[10, 3312],
 [10, 3312, 2492],
 [10, 3312, 2492, 3929],
 [10, 3312, 2492, 3929, 1280],
 [10, 3312, 2492, 3929, 1280, 81],
 [10, 3312, 2492, 3929, 1280, 81, 505],
 [10, 3312, 2492, 3929, 1280, 81, 505, 60],
 [10, 3312, 2492, 3929, 1280, 81, 505, 60, 147],
 [10, 3312, 2492, 3929, 1280, 81, 505, 60, 147, 2168],
 [10, 3312, 2492, 3929, 1280, 81, 505, 60, 147, 2168, 18169]]

We then want to *pad* our input sequences to make them all the same length.

In [11]:
# padding makes different inputs the same length as the longest input by adding 0's where words are missing in inputs
predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)
# E.g. with "My cat" and "The big dog" a 0 would be added in the front like "0 My cat"

## Create model

We then use the ```create_model()``` function created above to initialize a model, telling the model the length of sequences and the total size of the vocabulary.

In [12]:
model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 23, 10)            112650    
                                                                 
 lstm (LSTM)                 (None, 100)               44400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 11265)             1137765   


2023-03-22 13:28:48.180028: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-03-22 13:28:48.182795: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-03-22 13:28:48.185262: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

                                                                 
Total params: 1,294,815
Trainable params: 1,294,815
Non-trainable params: 0
_________________________________________________________________


Model training is exactly the same as last week, but instead of document labels, we're fitting the model to predict next word.

*NB!* This will take some time to train! It took me 35 minutes on UCloud 32xCPU.

In [13]:
# creating history of the model
history = model.fit(predictors, 
                    label, 
                    epochs=100,
                    batch_size=128, 
                    verbose=1)

Epoch 1/100


2023-03-22 13:28:56.409244: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-03-22 13:28:56.413034: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-03-22 13:28:56.415629: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

405/405 [==============================] - 34s 71ms/step - loss: 7.8848
Epoch 2/100
405/405 [==============================] - 27s 67ms/step - loss: 7.4818
Epoch 3/100
405/405 [==============================] - 27s 66ms/step - loss: 7.3958
Epoch 4/100
405/405 [==============================] - 30s 73ms/step - loss: 7.3057
Epoch 5/100
405/405 [==============================] - 29s 73ms/step - loss: 7.2025
Epoch 6/100
405/405 [==============================] - 27s 68ms/step - loss: 7.0930
Epoch 7/100
405/405 [==============================] - 25s 62ms/step - loss: 6.9781
Epoch 8/100
405/405 [==============================] - 25s 62ms/step - loss: 6.8568
Epoch 9/100
405/405 [==============================] - 26s 63ms/step - loss: 6.7294
Epoch 10/100
405/405 [==============================] - 25s 63ms/step - loss: 6.6030
Epoch 11/100
405/405 [==============================] - 26s 64ms/step - loss: 6.4755
Epoch 12/100
405/405 [==============================] - 24s 60ms/step - loss: 6.3505
E

When the model has trained, we can then use this to generate *new text*.

In [14]:
# print text based on the word/words and what the model has predicted would go along with it
print(generate_text("danish", 5, model, max_sequence_len)) # the 5 is the number of words we want to come after

2023-03-22 14:11:10.505500: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-03-22 14:11:10.507676: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-03-22 14:11:10.510464: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 0s 21ms/step
Danish Inventor Is Found Guilty In


In [26]:
# saving
import joblib
from joblib import dump
# saving trained model
dump(history, "../out/rnn_model.joblib")

['../out/rnn_model.joblib']

In [28]:
# load model
loaded_model = joblib.load("../out/rnn_model.joblib")
print(generate_text("danish", 5, model, max_sequence_len))

2023-03-22 14:26:42.620445: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-03-22 14:26:42.623448: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-03-22 14:26:42.625440: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 0s 21ms/step
Danish Inventor Is Found Guilty In


## Using pre-trained word embeddings

Instead of having the embedding layer as a trainable parameter, we can instead using a *pretrained word embedding* model like ```word2vec```.

In the following examples, we're using [GloVe embeddings](https://nlp.stanford.edu/projects/glove/). These are trained a little differently from ```word2vec``` but they behave in the same way.

We make use of already trained word embeddings, which are better than what we ever could train on a model ourselves.

In [42]:
# defining path
path_to_glove_file = os.path.join("../data/glove/")

# creating pre-trained word embedding index
embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

KeyboardInterrupt: 

We can define some variables that we're going to use later.

With hits and misses, we're counting how many words in the corpus vocabulary have a corresponding GloVe embedding; misses are the words which appear in our vocabulary but which do not have a GloVe embedding.

In [ ]:
num_tokens = total_words
embedding_dim = 100
hits = 0
misses = 0

In [ ]:
# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

In [ ]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer - notice that this is different
    model.add(Embedding(
            total_words,
            embedding_dim,
            embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
            trainable=False,
            input_length=input_len)
    )
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(500))
    model.add(Dropout(0.2))
    
    # Add Output Layer
    model.add(Dense(total_words, 
                    activation='softmax'))

    model.compile(loss='categorical_crossentropy', 
                    optimizer='adam')
    
    return model

In [ ]:
model = create_model(max_sequence_len, total_words)
model.summary()

In [ ]:
history = model.fit(predictors, 
                    label, 
                    epochs=100,
                    batch_size=128, 
                    verbose=1)

In [ ]:
print (generate_text("china", 30, model, max_sequence_len))